In [0]:
#!wget https://uaelxg.bn.files.1drv.com/y4m6LXxg-7eOXxzRU9IXUnf1XlbkQRLyAfwek0ViUe4JWDue6kQfX4S1nds_8YLl11gcp2CKL7lERNOEr1J49BV-R-wmnMOikf2q32UIXPMC3R02R139tW4r0nglP6UXCwgLlyXR0cnUT4wvKSNbTTXe_HMej6e-ai89rAMZmI4-LbXdxzv7OTVyK5JFRMC5-qw/fruits-360_dataset_2018_06_03.zip

In [0]:
# import zipfile
# zip_ref = zipfile.ZipFile('fruits-360_dataset_2018_06_03.zip', 'r')
# zip_ref.extractall('data2')
# zip_ref.close()

In [5]:
!ls

data2				     logbatch256.zip.zip  logs4
datalab				     logs		  logs5
fruits-360_dataset_2018_06_03.zip    logs2		  log.zip.zip
fruits-360_dataset_2018_06_03.zip.1  logs3


In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import glob
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [7]:
!ls


data2				     logbatch256.zip.zip  logs4
datalab				     logs		  logs5
fruits-360_dataset_2018_06_03.zip    logs2		  log.zip.zip
fruits-360_dataset_2018_06_03.zip.1  logs3


In [0]:
training_fruit_img = []
training_label = []
for dir_path in glob.glob("data2/fruits-360/Training/*"):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        training_fruit_img.append(image)
        training_label.append(img_label)
training_fruit_img = np.array(training_fruit_img)
training_label = np.array(training_label)
label_to_id = {v:k for k,v in enumerate(np.unique(training_label)) }
id_to_label = {v:k for k,v in label_to_id.items()}
training_label_id = np.array([label_to_id[i] for i in training_label])
validation_fruit_img=[]
validation_label =[]
for dir_path in glob.glob("data2/fruits-360/Validation/*"):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        validation_fruit_img.append(image)
        validation_label.append(img_label)
validation_fruit_img = np.array(validation_fruit_img)
validation_label = np.array(validation_label)
validation_label_id = np.array([label_to_id[i] for i in validation_label])

In [9]:
training_fruit_img.shape

(32426, 64, 64, 3)

In [0]:
def cnn(batch_size = 128, num_classes = 65, epochs = 12, numfilter1 = 32,y_train = training_label_id,
  y_test = validation_label_id, x_train = training_fruit_img.astype('float32'), x_test = validation_fruit_img.astype('float32'),kernel_size1=(3, 3),
        numfilter2 = 64, kernel_size2=(3, 3), activation='relu', log_dir='logs', optimizer=keras.optimizers.Adadelta() ):
  

  batch_size = batch_size
  num_classes = num_classes
  epochs = epochs

  # input image dimensions
  img_rows, img_cols = 64, 64
  input_shape = (img_rows,img_cols,3)


  y_train = training_label_id
  y_test = validation_label_id

  x_train = training_fruit_img.astype('float32')
  x_test = validation_fruit_img.astype('float32')
  x_train /= 255
  x_test /= 255
  print('x_train shape:', x_train.shape)
  print(x_train.shape[0], 'train samples')
  print(x_test.shape[0], 'test samples')

  # convert class vectors to binary class matrices
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)

  model = Sequential()
  model.add(Conv2D(numfilter1, kernel_size=kernel_size1,
                   activation=activation,
                   input_shape=input_shape))
  model.add(Conv2D(numfilter2, kernel_size2, activation=activation))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(128, activation=activation))
  model.add(Dropout(0.25))
  model.add(Dense(num_classes, activation='softmax'))

  from keras.callbacks import TensorBoard

  tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0,
                          write_graph=True, write_images=False)
  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
  model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard])
  score = model.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

In [11]:
!ls


data2				     logbatch256.zip.zip  logs4
datalab				     logs		  logs5
fruits-360_dataset_2018_06_03.zip    logs2		  log.zip.zip
fruits-360_dataset_2018_06_03.zip.1  logs3


In [12]:
!rm -r logsdefault

rm: cannot remove 'logsdefault': No such file or directory


In [13]:
cnn(batch_size = 128, num_classes = 65, epochs = 12, numfilter1 = 32,y_train = training_label_id,
  y_test = validation_label_id, x_train = training_fruit_img.astype('float32'), x_test = validation_fruit_img.astype('float32'),kernel_size1=(3, 3),
        numfilter2 = 64, kernel_size2=(3, 3), activation='relu', log_dir='logdefault', optimizer=keras.optimizers.Adadelta() )


x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/12
32426/32426 [==============================] - 27s 828us/step - loss: 1.8120 - acc: 0.5165 - val_loss: 0.3821 - val_acc: 0.8688
Epoch 2/12
15872/32426 [=============>................] - ETA: 11s - loss: 0.2194 - acc: 0.9296

32426/32426 [==============================] - 25s 774us/step - loss: 0.1704 - acc: 0.9443 - val_loss: 0.2134 - val_acc: 0.9272
Epoch 3/12
32426/32426 [==============================] - 25s 774us/step - loss: 0.0735 - acc: 0.9747 - val_loss: 0.1532 - val_acc: 0.9485
Epoch 4/12
11392/32426 [=========>....................] - ETA: 14s - loss: 0.0551 - acc: 0.9793

32426/32426 [==============================] - 25s 770us/step - loss: 0.0485 - acc: 0.9818 - val_loss: 0.1999 - val_acc: 0.9368
Epoch 5/12
32426/32426 [==============================] - 25s 773us/step - loss: 0.0383 - acc: 0.9848 - val_loss: 0.1408 - val_acc: 0.9528
Epoch 6/12
10368/32426 [========>.....................] - ETA: 15s - loss: 0.0344 - acc: 0.9859

32426/32426 [==============================] - 25s 775us/step - loss: 0.0299 - acc: 0.9873 - val_loss: 0.2164 - val_acc: 0.9408
Epoch 7/12
32426/32426 [==============================] - 25s 772us/step - loss: 0.0280 - acc: 0.9874 - val_loss: 0.1961 - val_acc: 0.9451
Epoch 8/12
10112/32426 [========>.....................] - ETA: 15s - loss: 0.0221 - acc: 0.9899

32426/32426 [==============================] - 25s 772us/step - loss: 0.0247 - acc: 0.9889 - val_loss: 0.1432 - val_acc: 0.9535
Epoch 9/12
32426/32426 [==============================] - 25s 774us/step - loss: 0.0221 - acc: 0.9901 - val_loss: 0.1391 - val_acc: 0.9586
Epoch 10/12
10112/32426 [========>.....................] - ETA: 15s - loss: 0.0218 - acc: 0.9907

32426/32426 [==============================] - 25s 771us/step - loss: 0.0216 - acc: 0.9903 - val_loss: 0.2070 - val_acc: 0.9474
Epoch 11/12
32426/32426 [==============================] - 25s 775us/step - loss: 0.0203 - acc: 0.9898 - val_loss: 0.1681 - val_acc: 0.9599
Epoch 12/12
 9984/32426 [========>.....................] - ETA: 15s - loss: 0.0192 - acc: 0.9907

32426/32426 [==============================] - 25s 775us/step - loss: 0.0197 - acc: 0.9902 - val_loss: 0.1615 - val_acc: 0.9568
Test loss: 0.16153730784703177
Test accuracy: 0.9568008804916078


In [16]:
cnn(batch_size = 128, num_classes = 65, epochs = 12, numfilter1 = 32,y_train = training_label_id,
  y_test = validation_label_id, x_train = training_fruit_img.astype('float32'), x_test = validation_fruit_img.astype('float32'),kernel_size1=(5, 5),
        numfilter2 = 64, kernel_size2=(5, 5), activation='relu', log_dir='logs', optimizer=keras.optimizers.Adadelta() )

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/12
32426/32426 [==============================] - 26s 794us/step - loss: 1.7482 - acc: 0.5414 - val_loss: 0.4628 - val_acc: 0.8619
Epoch 2/12
15872/32426 [=============>................] - ETA: 11s - loss: 0.1675 - acc: 0.9463

32426/32426 [==============================] - 25s 759us/step - loss: 0.1303 - acc: 0.9569 - val_loss: 0.2705 - val_acc: 0.9198
Epoch 3/12
32426/32426 [==============================] - 25s 761us/step - loss: 0.0751 - acc: 0.9742 - val_loss: 0.4265 - val_acc: 0.8802
Epoch 4/12
11392/32426 [=========>....................] - ETA: 14s - loss: 0.0510 - acc: 0.9790

32426/32426 [==============================] - 25s 764us/step - loss: 0.0446 - acc: 0.9827 - val_loss: 0.2558 - val_acc: 0.9295
Epoch 5/12
32426/32426 [==============================] - 25s 763us/step - loss: 0.0416 - acc: 0.9848 - val_loss: 0.2272 - val_acc: 0.9393
Epoch 6/12
10368/32426 [========>.....................] - ETA: 15s - loss: 0.0247 - acc: 0.9877

32426/32426 [==============================] - 25s 760us/step - loss: 0.0264 - acc: 0.9881 - val_loss: 0.3103 - val_acc: 0.9303
Epoch 7/12
32426/32426 [==============================] - 25s 761us/step - loss: 0.0234 - acc: 0.9902 - val_loss: 0.1877 - val_acc: 0.9546
Epoch 8/12
10112/32426 [========>.....................] - ETA: 15s - loss: 0.0293 - acc: 0.9869

32426/32426 [==============================] - 25s 762us/step - loss: 0.0271 - acc: 0.9893 - val_loss: 0.2094 - val_acc: 0.9494
Epoch 9/12
32426/32426 [==============================] - 25s 763us/step - loss: 0.0220 - acc: 0.9896 - val_loss: 0.2132 - val_acc: 0.9508
Epoch 10/12
10112/32426 [========>.....................] - ETA: 15s - loss: 0.0180 - acc: 0.9911

32426/32426 [==============================] - 25s 760us/step - loss: 0.0195 - acc: 0.9904 - val_loss: 0.2205 - val_acc: 0.9535
Epoch 11/12
32426/32426 [==============================] - 25s 761us/step - loss: 0.0476 - acc: 0.9865 - val_loss: 0.2285 - val_acc: 0.9515
Epoch 12/12
 9984/32426 [========>.....................] - ETA: 15s - loss: 0.0177 - acc: 0.9911

32426/32426 [==============================] - 25s 762us/step - loss: 0.0203 - acc: 0.9907 - val_loss: 0.3477 - val_acc: 0.9261
Test loss: 0.34770047377510244
Test accuracy: 0.9260753920939191


In [17]:
cnn(batch_size = 256, num_classes = 65, epochs = 30, numfilter1 = 32,
        kernel_size1=(3, 3), numfilter2 = 64, kernel_size2=(3, 3), activation='relu', log_dir='logbatch256')

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/30
32426/32426 [==============================] - 23s 715us/step - loss: 3.0600 - acc: 0.2150 - val_loss: 1.7024 - val_acc: 0.4870
Epoch 2/30
32426/32426 [==============================] - 22s 673us/step - loss: 0.8172 - acc: 0.7478 - val_loss: 0.5998 - val_acc: 0.7957
Epoch 3/30
13568/32426 [===========>..................] - ETA: 11s - loss: 0.3289 - acc: 0.8964

32426/32426 [==============================] - 22s 673us/step - loss: 0.2694 - acc: 0.9137 - val_loss: 0.3874 - val_acc: 0.8766
Epoch 4/30
32426/32426 [==============================] - 22s 672us/step - loss: 0.1515 - acc: 0.9504 - val_loss: 0.1802 - val_acc: 0.9385
Epoch 5/30
32426/32426 [==============================] - 22s 673us/step - loss: 0.1090 - acc: 0.9628 - val_loss: 0.1784 - val_acc: 0.9311
Epoch 6/30
 9728/32426 [========>.....................] - ETA: 13s - loss: 0.0776 - acc: 0.9728

32426/32426 [==============================] - 22s 673us/step - loss: 0.0761 - acc: 0.9723 - val_loss: 0.1591 - val_acc: 0.9456
Epoch 7/30
32426/32426 [==============================] - 22s 675us/step - loss: 0.0648 - acc: 0.9765 - val_loss: 0.1862 - val_acc: 0.9473
Epoch 8/30
32426/32426 [==============================] - 22s 676us/step - loss: 0.0544 - acc: 0.9800 - val_loss: 0.1227 - val_acc: 0.9624
Epoch 9/30
 9216/32426 [=======>......................] - ETA: 14s - loss: 0.0521 - acc: 0.9795

32426/32426 [==============================] - 22s 675us/step - loss: 0.0474 - acc: 0.9817 - val_loss: 0.1924 - val_acc: 0.9486
Epoch 10/30
32426/32426 [==============================] - 22s 673us/step - loss: 0.0415 - acc: 0.9840 - val_loss: 0.1518 - val_acc: 0.9542
Epoch 11/30
32426/32426 [==============================] - 22s 673us/step - loss: 0.0423 - acc: 0.9828 - val_loss: 0.1642 - val_acc: 0.9501
Epoch 12/30
 8960/32426 [=======>......................] - ETA: 14s - loss: 0.0422 - acc: 0.9855

32426/32426 [==============================] - 22s 675us/step - loss: 0.0393 - acc: 0.9845 - val_loss: 0.1441 - val_acc: 0.9584
Epoch 13/30
32426/32426 [==============================] - 22s 673us/step - loss: 0.0327 - acc: 0.9862 - val_loss: 0.1638 - val_acc: 0.9489
Epoch 14/30
32426/32426 [==============================] - 22s 672us/step - loss: 0.0345 - acc: 0.9854 - val_loss: 0.1331 - val_acc: 0.9588
Epoch 15/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0301 - acc: 0.9874

...

32426/32426 [==============================] - 22s 672us/step - loss: 0.0262 - acc: 0.9882 - val_loss: 0.1316 - val_acc: 0.9612
Epoch 19/30
32426/32426 [==============================] - 22s 674us/step - loss: 0.0262 - acc: 0.9881 - val_loss: 0.1439 - val_acc: 0.9609
Epoch 20/30
32426/32426 [==============================] - 22s 672us/step - loss: 0.0254 - acc: 0.9876 - val_loss: 0.1676 - val_acc: 0.9669
Epoch 21/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0292 - acc: 0.9876

32426/32426 [==============================] - 22s 673us/step - loss: 0.0256 - acc: 0.9879 - val_loss: 0.1454 - val_acc: 0.9581
Epoch 22/30
32426/32426 [==============================] - 22s 672us/step - loss: 0.0252 - acc: 0.9885 - val_loss: 0.1445 - val_acc: 0.9611
Epoch 23/30
32426/32426 [==============================] - 22s 673us/step - loss: 0.0257 - acc: 0.9877 - val_loss: 0.1470 - val_acc: 0.9654
Epoch 24/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0259 - acc: 0.9877

32426/32426 [==============================] - 22s 672us/step - loss: 0.0247 - acc: 0.9882 - val_loss: 0.1386 - val_acc: 0.9611
Epoch 25/30
32426/32426 [==============================] - 22s 672us/step - loss: 0.0225 - acc: 0.9897 - val_loss: 0.1211 - val_acc: 0.9698
Epoch 26/30
32426/32426 [==============================] - 22s 672us/step - loss: 0.0230 - acc: 0.9887 - val_loss: 0.1525 - val_acc: 0.9654
Epoch 27/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0167 - acc: 0.9916

32426/32426 [==============================] - 22s 671us/step - loss: 0.0227 - acc: 0.9891 - val_loss: 0.1501 - val_acc: 0.9681
Epoch 28/30
32426/32426 [==============================] - 22s 671us/step - loss: 0.0221 - acc: 0.9890 - val_loss: 0.1757 - val_acc: 0.9606
Epoch 29/30
32426/32426 [==============================] - 22s 672us/step - loss: 0.0223 - acc: 0.9893 - val_loss: 0.1379 - val_acc: 0.9716
Epoch 30/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0200 - acc: 0.9898

32426/32426 [==============================] - 22s 672us/step - loss: 0.0217 - acc: 0.9891 - val_loss: 0.2352 - val_acc: 0.9465
Test loss: 0.23515295318411505
Test accuracy: 0.9465284784004403


In [18]:
cnn(batch_size = 256, num_classes = 65, epochs = 30, numfilter1 = 32,
        kernel_size1=(5, 5), numfilter2 = 64, kernel_size2=(3, 3), activation='relu', log_dir='log53')

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/30
32426/32426 [==============================] - 22s 681us/step - loss: 2.6165 - acc: 0.3228 - val_loss: 0.7965 - val_acc: 0.7755
Epoch 2/30
32426/32426 [==============================] - 21s 637us/step - loss: 0.4506 - acc: 0.8595 - val_loss: 0.3113 - val_acc: 0.8920
Epoch 3/30
13568/32426 [===========>..................] - ETA: 10s - loss: 0.2275 - acc: 0.9278

32426/32426 [==============================] - 21s 636us/step - loss: 0.1606 - acc: 0.9468 - val_loss: 0.2483 - val_acc: 0.9228
Epoch 4/30
32426/32426 [==============================] - 21s 638us/step - loss: 0.1021 - acc: 0.9653 - val_loss: 0.2225 - val_acc: 0.9338
Epoch 5/30
32426/32426 [==============================] - 21s 637us/step - loss: 0.0622 - acc: 0.9779 - val_loss: 0.1515 - val_acc: 0.9486
Epoch 6/30
 9728/32426 [========>.....................] - ETA: 12s - loss: 0.0450 - acc: 0.9833

32426/32426 [==============================] - 21s 637us/step - loss: 0.1753 - acc: 0.9573 - val_loss: 0.1630 - val_acc: 0.9504
Epoch 7/30
32426/32426 [==============================] - 21s 637us/step - loss: 0.0387 - acc: 0.9856 - val_loss: 0.1581 - val_acc: 0.9518
Epoch 8/30
32426/32426 [==============================] - 21s 635us/step - loss: 0.0407 - acc: 0.9839 - val_loss: 0.2096 - val_acc: 0.9360
Epoch 9/30
 9216/32426 [=======>......................] - ETA: 13s - loss: 0.1823 - acc: 0.9531

32426/32426 [==============================] - 21s 636us/step - loss: 0.0716 - acc: 0.9783 - val_loss: 0.1777 - val_acc: 0.9527
Epoch 10/30
32426/32426 [==============================] - 21s 636us/step - loss: 0.0285 - acc: 0.9871 - val_loss: 0.4723 - val_acc: 0.8835
Epoch 11/30
32426/32426 [==============================] - 21s 636us/step - loss: 0.0781 - acc: 0.9754 - val_loss: 0.1322 - val_acc: 0.9576
Epoch 12/30
 8960/32426 [=======>......................] - ETA: 13s - loss: 0.0262 - acc: 0.9867

32426/32426 [==============================] - 21s 636us/step - loss: 0.0263 - acc: 0.9879 - val_loss: 0.1847 - val_acc: 0.9495
Epoch 13/30
32426/32426 [==============================] - 21s 638us/step - loss: 0.0230 - acc: 0.9886 - val_loss: 0.1760 - val_acc: 0.9529
Epoch 14/30
32426/32426 [==============================] - 21s 636us/step - loss: 0.0238 - acc: 0.9896 - val_loss: 0.2094 - val_acc: 0.9494
Epoch 15/30
 8704/32426 [=======>......................] - ETA: 13s - loss: 0.0221 - acc: 0.9897

...

32426/32426 [==============================] - 21s 638us/step - loss: 0.0196 - acc: 0.9907 - val_loss: 0.1753 - val_acc: 0.9512
Epoch 19/30
32426/32426 [==============================] - 21s 638us/step - loss: 0.0172 - acc: 0.9911 - val_loss: 0.1784 - val_acc: 0.9516
Epoch 20/30
32426/32426 [==============================] - 21s 637us/step - loss: 0.0176 - acc: 0.9906 - val_loss: 0.2122 - val_acc: 0.9505
Epoch 21/30
 8704/32426 [=======>......................] - ETA: 13s - loss: 0.0156 - acc: 0.9913

32426/32426 [==============================] - 21s 636us/step - loss: 0.0182 - acc: 0.9904 - val_loss: 0.1734 - val_acc: 0.9575
Epoch 22/30
32426/32426 [==============================] - 21s 638us/step - loss: 0.0187 - acc: 0.9896 - val_loss: 0.2500 - val_acc: 0.9423
Epoch 23/30
32426/32426 [==============================] - 21s 637us/step - loss: 0.0165 - acc: 0.9908 - val_loss: 0.2095 - val_acc: 0.9539
Epoch 24/30
 8704/32426 [=======>......................] - ETA: 13s - loss: 0.0184 - acc: 0.9913

32426/32426 [==============================] - 21s 637us/step - loss: 0.0180 - acc: 0.9903 - val_loss: 0.1757 - val_acc: 0.9614
Epoch 25/30
32426/32426 [==============================] - 21s 637us/step - loss: 0.0172 - acc: 0.9903 - val_loss: 0.1682 - val_acc: 0.9561
Epoch 26/30
32426/32426 [==============================] - 21s 636us/step - loss: 0.0155 - acc: 0.9912 - val_loss: 0.1748 - val_acc: 0.9574
Epoch 27/30
 8704/32426 [=======>......................] - ETA: 13s - loss: 0.0153 - acc: 0.9921

32426/32426 [==============================] - 21s 637us/step - loss: 0.0162 - acc: 0.9911 - val_loss: 0.1801 - val_acc: 0.9509
Epoch 28/30
32426/32426 [==============================] - 21s 638us/step - loss: 0.0167 - acc: 0.9908 - val_loss: 0.1752 - val_acc: 0.9580
Epoch 29/30
32426/32426 [==============================] - 21s 637us/step - loss: 0.0154 - acc: 0.9918 - val_loss: 0.1968 - val_acc: 0.9525
Epoch 30/30
 8704/32426 [=======>......................] - ETA: 13s - loss: 0.0148 - acc: 0.9910

32426/32426 [==============================] - 21s 636us/step - loss: 0.2149 - acc: 0.9598 - val_loss: 0.2105 - val_acc: 0.9452
Test loss: 0.2105261381798417
Test accuracy: 0.9452444281390443


In [19]:
cnn(batch_size = 256, num_classes = 65, epochs = 30, numfilter1 = 32,
        kernel_size1=(3, 3), numfilter2 = 64, kernel_size2=(5, 5), activation='relu', log_dir='log35')

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/30
32426/32426 [==============================] - 25s 765us/step - loss: 2.3708 - acc: 0.3881 - val_loss: 0.8564 - val_acc: 0.7515
Epoch 2/30
32426/32426 [==============================] - 23s 701us/step - loss: 0.2853 - acc: 0.9113 - val_loss: 0.4194 - val_acc: 0.8787
Epoch 3/30
13568/32426 [===========>..................] - ETA: 11s - loss: 0.1110 - acc: 0.9646

32426/32426 [==============================] - 23s 702us/step - loss: 0.0885 - acc: 0.9712 - val_loss: 0.2370 - val_acc: 0.9208
Epoch 4/30
32426/32426 [==============================] - 23s 701us/step - loss: 0.0620 - acc: 0.9787 - val_loss: 0.1939 - val_acc: 0.9323
Epoch 5/30
32426/32426 [==============================] - 23s 699us/step - loss: 0.0338 - acc: 0.9875 - val_loss: 0.1848 - val_acc: 0.9361
Epoch 6/30
 9728/32426 [========>.....................] - ETA: 14s - loss: 0.0279 - acc: 0.9876

32426/32426 [==============================] - 23s 698us/step - loss: 0.0300 - acc: 0.9873 - val_loss: 0.1806 - val_acc: 0.9435
Epoch 7/30
32426/32426 [==============================] - 23s 699us/step - loss: 0.0256 - acc: 0.9894 - val_loss: 0.1914 - val_acc: 0.9427
Epoch 8/30
32426/32426 [==============================] - 23s 698us/step - loss: 0.0253 - acc: 0.9891 - val_loss: 0.1908 - val_acc: 0.9491
Epoch 9/30
 9216/32426 [=======>......................] - ETA: 14s - loss: 0.0182 - acc: 0.9903

32426/32426 [==============================] - 23s 698us/step - loss: 0.0205 - acc: 0.9899 - val_loss: 0.1656 - val_acc: 0.9514
Epoch 10/30
32426/32426 [==============================] - 23s 698us/step - loss: 0.0199 - acc: 0.9904 - val_loss: 0.2555 - val_acc: 0.9331
Epoch 11/30
32426/32426 [==============================] - 23s 699us/step - loss: 0.0170 - acc: 0.9912 - val_loss: 0.1761 - val_acc: 0.9504
Epoch 12/30
 8960/32426 [=======>......................] - ETA: 14s - loss: 0.0214 - acc: 0.9900

32426/32426 [==============================] - 23s 699us/step - loss: 0.0186 - acc: 0.9908 - val_loss: 0.2124 - val_acc: 0.9449
Epoch 13/30
32426/32426 [==============================] - 23s 699us/step - loss: 0.0162 - acc: 0.9914 - val_loss: 0.2306 - val_acc: 0.9430
Epoch 14/30
32426/32426 [==============================] - 23s 697us/step - loss: 0.0169 - acc: 0.9912 - val_loss: 0.2105 - val_acc: 0.9401
Epoch 15/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0304 - acc: 0.9868

...

32426/32426 [==============================] - 23s 698us/step - loss: 0.0165 - acc: 0.9914 - val_loss: 0.1971 - val_acc: 0.9487
Epoch 19/30
32426/32426 [==============================] - 23s 698us/step - loss: 0.0142 - acc: 0.9919 - val_loss: 0.1784 - val_acc: 0.9550
Epoch 20/30
32426/32426 [==============================] - 23s 698us/step - loss: 0.0165 - acc: 0.9912 - val_loss: 0.1859 - val_acc: 0.9532
Epoch 21/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0126 - acc: 0.9933

32426/32426 [==============================] - 23s 698us/step - loss: 0.0137 - acc: 0.9926 - val_loss: 0.1849 - val_acc: 0.9487
Epoch 22/30
32426/32426 [==============================] - 23s 699us/step - loss: 0.0164 - acc: 0.9913 - val_loss: 0.2329 - val_acc: 0.9489
Epoch 23/30
32426/32426 [==============================] - 23s 698us/step - loss: 0.0139 - acc: 0.9925 - val_loss: 0.1710 - val_acc: 0.9607
Epoch 24/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0134 - acc: 0.9930

32426/32426 [==============================] - 23s 700us/step - loss: 0.0147 - acc: 0.9920 - val_loss: 0.1866 - val_acc: 0.9558
Epoch 25/30
32426/32426 [==============================] - 23s 700us/step - loss: 0.0124 - acc: 0.9927 - val_loss: 0.1811 - val_acc: 0.9564
Epoch 26/30
32426/32426 [==============================] - 23s 699us/step - loss: 0.0133 - acc: 0.9920 - val_loss: 0.1735 - val_acc: 0.9567
Epoch 27/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0131 - acc: 0.9918

32426/32426 [==============================] - 23s 699us/step - loss: 0.0131 - acc: 0.9921 - val_loss: 0.2946 - val_acc: 0.9336
Epoch 28/30
32426/32426 [==============================] - 23s 697us/step - loss: 0.0124 - acc: 0.9923 - val_loss: 0.1954 - val_acc: 0.9518
Epoch 29/30
32426/32426 [==============================] - 23s 698us/step - loss: 0.0132 - acc: 0.9924 - val_loss: 0.2416 - val_acc: 0.9481
Epoch 30/30
 8704/32426 [=======>......................] - ETA: 14s - loss: 0.0146 - acc: 0.9910

32426/32426 [==============================] - 23s 698us/step - loss: 0.0129 - acc: 0.9919 - val_loss: 0.2164 - val_acc: 0.9536
Test loss: 0.21640918854984553
Test accuracy: 0.953590754838118


In [20]:
cnn(batch_size = 256, num_classes = 65, epochs = 30, numfilter1 = 64,
        kernel_size1=(3, 3), numfilter2 = 64, kernel_size2=(5, 5), activation='relu', log_dir='log64')

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/30
32426/32426 [==============================] - 34s 1ms/step - loss: 2.4371 - acc: 0.3768 - val_loss: 0.7221 - val_acc: 0.7933
Epoch 2/30
32426/32426 [==============================] - 30s 937us/step - loss: 0.2717 - acc: 0.9171 - val_loss: 0.3301 - val_acc: 0.8983
Epoch 3/30
13568/32426 [===========>..................] - ETA: 15s - loss: 0.0833 - acc: 0.9712

32426/32426 [==============================] - 30s 940us/step - loss: 0.1238 - acc: 0.9617 - val_loss: 0.1922 - val_acc: 0.9390
Epoch 4/30
32426/32426 [==============================] - 30s 939us/step - loss: 0.0413 - acc: 0.9849 - val_loss: 0.3034 - val_acc: 0.8957
Epoch 5/30
32426/32426 [==============================] - 31s 941us/step - loss: 0.0504 - acc: 0.9821 - val_loss: 0.1829 - val_acc: 0.9456
Epoch 6/30
 9472/32426 [=======>......................] - ETA: 19s - loss: 0.0344 - acc: 0.9863

32426/32426 [==============================] - 30s 938us/step - loss: 0.0485 - acc: 0.9835 - val_loss: 0.1532 - val_acc: 0.9513
Epoch 7/30
32426/32426 [==============================] - 30s 939us/step - loss: 0.0294 - acc: 0.9885 - val_loss: 0.1664 - val_acc: 0.9470
Epoch 8/30
32426/32426 [==============================] - 30s 938us/step - loss: 0.0210 - acc: 0.9907 - val_loss: 0.2324 - val_acc: 0.9403
Epoch 9/30
 8704/32426 [=======>......................] - ETA: 20s - loss: 0.0170 - acc: 0.9915

32426/32426 [==============================] - 30s 938us/step - loss: 0.1037 - acc: 0.9725 - val_loss: 0.1869 - val_acc: 0.9434
Epoch 10/30
32426/32426 [==============================] - 30s 937us/step - loss: 0.0174 - acc: 0.9917 - val_loss: 0.2022 - val_acc: 0.9496
Epoch 11/30
32426/32426 [==============================] - 30s 938us/step - loss: 0.0190 - acc: 0.9911 - val_loss: 0.1729 - val_acc: 0.9568
Epoch 12/30
 8192/32426 [======>.......................] - ETA: 20s - loss: 0.0156 - acc: 0.9917

32426/32426 [==============================] - 30s 936us/step - loss: 0.0152 - acc: 0.9922 - val_loss: 0.1765 - val_acc: 0.9520
Epoch 13/30
32426/32426 [==============================] - 30s 936us/step - loss: 0.0159 - acc: 0.9914 - val_loss: 0.1695 - val_acc: 0.9566
Epoch 14/30
32426/32426 [==============================] - 30s 936us/step - loss: 0.0177 - acc: 0.9909 - val_loss: 0.3203 - val_acc: 0.9375
Epoch 15/30
 8192/32426 [======>.......................] - ETA: 20s - loss: 0.0205 - acc: 0.9894

...

32426/32426 [==============================] - 30s 935us/step - loss: 0.1968 - acc: 0.9628 - val_loss: 0.1973 - val_acc: 0.9414
Epoch 19/30
32426/32426 [==============================] - 30s 935us/step - loss: 0.0160 - acc: 0.9917 - val_loss: 0.2426 - val_acc: 0.9400
Epoch 20/30
32426/32426 [==============================] - 30s 936us/step - loss: 0.0154 - acc: 0.9916 - val_loss: 0.2071 - val_acc: 0.9534
Epoch 21/30
 8192/32426 [======>.......................] - ETA: 20s - loss: 0.0125 - acc: 0.9927

32426/32426 [==============================] - 30s 936us/step - loss: 0.0139 - acc: 0.9924 - val_loss: 0.2006 - val_acc: 0.9460
Epoch 22/30
32426/32426 [==============================] - 30s 934us/step - loss: 0.0138 - acc: 0.9922 - val_loss: 0.1926 - val_acc: 0.9529
Epoch 23/30
32426/32426 [==============================] - 30s 933us/step - loss: 0.0161 - acc: 0.9916 - val_loss: 0.2154 - val_acc: 0.9443
Epoch 24/30
 8192/32426 [======>.......................] - ETA: 20s - loss: 0.0128 - acc: 0.9924

32426/32426 [==============================] - 30s 934us/step - loss: 0.0119 - acc: 0.9932 - val_loss: 0.2001 - val_acc: 0.9497
Epoch 25/30
32426/32426 [==============================] - 30s 934us/step - loss: 0.0121 - acc: 0.9928 - val_loss: 0.2399 - val_acc: 0.9546
Epoch 26/30
32426/32426 [==============================] - 30s 932us/step - loss: 0.0133 - acc: 0.9918 - val_loss: 0.1992 - val_acc: 0.9543
Epoch 27/30
 8192/32426 [======>.......................] - ETA: 20s - loss: 0.0104 - acc: 0.9929

32426/32426 [==============================] - 30s 933us/step - loss: 0.0120 - acc: 0.9931 - val_loss: 0.2395 - val_acc: 0.9555
Epoch 28/30
32426/32426 [==============================] - 30s 934us/step - loss: 0.0124 - acc: 0.9921 - val_loss: 0.1671 - val_acc: 0.9586
Epoch 29/30
32426/32426 [==============================] - 30s 933us/step - loss: 0.0117 - acc: 0.9931 - val_loss: 0.1953 - val_acc: 0.9534
Epoch 30/30
 8192/32426 [======>.......................] - ETA: 20s - loss: 0.0108 - acc: 0.9924

32426/32426 [==============================] - 30s 934us/step - loss: 0.0127 - acc: 0.9923 - val_loss: 0.1880 - val_acc: 0.9591
Test loss: 0.1879547352963785
Test accuracy: 0.9590938273869577


In [21]:
cnn(batch_size = 256, num_classes = 65, epochs = 30, numfilter1 = 32,
        kernel_size1=(3, 3), numfilter2 = 32, kernel_size2=(3, 3), activation='relu', log_dir='log64')

x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/30
32426/32426 [==============================] - 15s 449us/step - loss: 2.3359 - acc: 0.3848 - val_loss: 0.6952 - val_acc: 0.8026
Epoch 2/30
32426/32426 [==============================] - 14s 422us/step - loss: 0.3266 - acc: 0.9008 - val_loss: 0.2689 - val_acc: 0.8999
Epoch 3/30
13568/32426 [===========>..................] - ETA: 7s - loss: 0.1144 - acc: 0.9644

32426/32426 [==============================] - 14s 423us/step - loss: 0.0936 - acc: 0.9701 - val_loss: 0.1992 - val_acc: 0.9366
Epoch 4/30
32426/32426 [==============================] - 14s 421us/step - loss: 0.0883 - acc: 0.9729 - val_loss: 0.1770 - val_acc: 0.9440
Epoch 5/30
32426/32426 [==============================] - 14s 423us/step - loss: 0.0345 - acc: 0.9874 - val_loss: 0.1575 - val_acc: 0.9454
Epoch 6/30
 9728/32426 [========>.....................] - ETA: 8s - loss: 0.0410 - acc: 0.9857

32426/32426 [==============================] - 14s 423us/step - loss: 0.0330 - acc: 0.9871 - val_loss: 0.1682 - val_acc: 0.9418
Epoch 7/30
32426/32426 [==============================] - 14s 424us/step - loss: 0.0278 - acc: 0.9886 - val_loss: 0.1402 - val_acc: 0.9522
Epoch 8/30
32426/32426 [==============================] - 14s 424us/step - loss: 0.0206 - acc: 0.9903 - val_loss: 0.1507 - val_acc: 0.9476
Epoch 9/30
 8448/32426 [======>.......................] - ETA: 9s - loss: 0.0203 - acc: 0.9906

32426/32426 [==============================] - 14s 422us/step - loss: 0.0222 - acc: 0.9897 - val_loss: 0.1649 - val_acc: 0.9438
Epoch 10/30
32426/32426 [==============================] - 14s 424us/step - loss: 0.0198 - acc: 0.9908 - val_loss: 0.1610 - val_acc: 0.9510
Epoch 11/30
32426/32426 [==============================] - 14s 423us/step - loss: 0.0176 - acc: 0.9916 - val_loss: 0.1295 - val_acc: 0.9547
Epoch 12/30
 8192/32426 [======>.......................] - ETA: 9s - loss: 0.0208 - acc: 0.9904

32426/32426 [==============================] - 14s 424us/step - loss: 0.0179 - acc: 0.9908 - val_loss: 0.1761 - val_acc: 0.9510
Epoch 13/30
32426/32426 [==============================] - 14s 424us/step - loss: 0.0164 - acc: 0.9916 - val_loss: 0.1313 - val_acc: 0.9609
Epoch 14/30
32426/32426 [==============================] - 14s 421us/step - loss: 0.0162 - acc: 0.9915 - val_loss: 0.2254 - val_acc: 0.9484
Epoch 15/30
 8704/32426 [=======>......................] - ETA: 8s - loss: 0.0154 - acc: 0.9915

...

32426/32426 [==============================] - 14s 424us/step - loss: 0.0139 - acc: 0.9922 - val_loss: 0.1609 - val_acc: 0.9649
Epoch 19/30
32426/32426 [==============================] - 14s 422us/step - loss: 0.0140 - acc: 0.9918 - val_loss: 0.1532 - val_acc: 0.9613
Epoch 20/30
32426/32426 [==============================] - 14s 425us/step - loss: 0.0138 - acc: 0.9923 - val_loss: 0.1518 - val_acc: 0.9596
Epoch 21/30
 6656/32426 [=====>........................] - ETA: 9s - loss: 0.0118 - acc: 0.9943

32426/32426 [==============================] - 14s 422us/step - loss: 0.0125 - acc: 0.9929 - val_loss: 0.1475 - val_acc: 0.9616
Epoch 22/30
32426/32426 [==============================] - 14s 422us/step - loss: 0.0149 - acc: 0.9920 - val_loss: 0.1685 - val_acc: 0.9653
Epoch 23/30
32426/32426 [==============================] - 14s 421us/step - loss: 0.0137 - acc: 0.9924 - val_loss: 0.1715 - val_acc: 0.9623
Epoch 24/30
 8192/32426 [======>.......................] - ETA: 9s - loss: 0.0125 - acc: 0.9935

32426/32426 [==============================] - 14s 423us/step - loss: 0.0131 - acc: 0.9928 - val_loss: 0.1458 - val_acc: 0.9568
Epoch 25/30
32426/32426 [==============================] - 14s 424us/step - loss: 0.0139 - acc: 0.9924 - val_loss: 0.1497 - val_acc: 0.9592
Epoch 26/30
32426/32426 [==============================] - 14s 423us/step - loss: 0.0129 - acc: 0.9924 - val_loss: 0.1982 - val_acc: 0.9599
Epoch 27/30
 7680/32426 [======>.......................] - ETA: 9s - loss: 0.0111 - acc: 0.9932

32426/32426 [==============================] - 14s 423us/step - loss: 0.0120 - acc: 0.9925 - val_loss: 0.1748 - val_acc: 0.9589
Epoch 28/30
32426/32426 [==============================] - 14s 422us/step - loss: 0.0136 - acc: 0.9919 - val_loss: 0.1454 - val_acc: 0.9626
Epoch 29/30
32426/32426 [==============================] - 14s 424us/step - loss: 0.0122 - acc: 0.9922 - val_loss: 0.1765 - val_acc: 0.9622
Epoch 30/30
 7936/32426 [======>.......................] - ETA: 9s - loss: 0.0307 - acc: 0.9887

32426/32426 [==============================] - 14s 423us/step - loss: 0.0174 - acc: 0.9915 - val_loss: 0.1701 - val_acc: 0.9628
Test loss: 0.17007495958431634
Test accuracy: 0.9627625424195175


In [16]:
!ls

data2	 fruits-360_dataset_2018_06_03.zip    logs
datalab  fruits-360_dataset_2018_06_03.zip.1


In [27]:
# import shutil
# shutil.make_archive('logbatch256.zip', 'zip', 'logs4')

'/content/logbatch256.zip.zip'

In [0]:
# from google.colab import files
# files.download('/content/logbatch256.zip.zip')  # from colab to browser download

In [0]:
# from sklearn.ensemble import RandomForestClassifier
# import numpy as np
# import pandas as pd

# rf = RandomForestClassifier(n_estimators=100)
# rf.fit(x_train.reshape(x_train.shape[0],64*64*3), y_train)
# pred = rf.predict(x_test.reshape(x_test.shape[0],64*64*3))


# from sklearn.metrics import accuracy_score



In [0]:
# print(accuracy_score(y_test, pred))